<a href="https://colab.research.google.com/github/LightYe4r/stock_data_analysis-SDA-/blob/master/%EC%9D%B4%EB%B3%B4%EB%A6%BC_%EC%A3%BC%EC%8B%9D%EC%B0%A8%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

아래 설치 후 런타임 재시작(세션 재시작)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 네이버 증권에서 반도체 관련 종목 가져오기

In [ ]:
stock = {}
with open('/content/drive/MyDrive/Colab Notebooks/반도체주식/네이버증권_반도체종목.txt','r') as f:
  lines = f.readlines()
  print(lines)
  for idx, line in enumerate(lines[::6]):
    stock[line.strip()] = lines[idx*6+1].strip()
print(stock)

In [ ]:
print(len(stock))

In [ ]:
!pip install finance-datareader
!pip install beautifulsoup4
!pip install matplotlib
!pip install pandas_ta
!pip install plotly==5.11.0

In [ ]:
import FinanceDataReader as fdr
import pandas_ta as ta
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
from matplotlib import gridspec
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.graph_objects as px
from plotly.subplots import make_subplots

In [ ]:
krx = fdr.StockListing('KRX')

In [ ]:
buttons = []
for val in stock.keys():
  button = dict()
  button['args'] = ['type',val]
  button['label'] = val
  button['method'] = 'update'
  buttons.append(button)

In [ ]:
def get_stockdata(stock_code):
  stock_data = fdr.DataReader(stock_code)
  return stock_data

def match_dfshape(df, start_date, end_date):
    # 주어진 기간 동안의 모든 날짜 생성
    expected_dates = pd.date_range(start=start_date, end=end_date)

    # 주식 DataFrame에 없는 날짜에 대한 데이터 생성
    missing_dates_data = pd.DataFrame(index=expected_dates, columns=df.columns)

    # 주식 DataFrame과 결합하여 결측치를 채움
    filled_df = pd.concat([df, missing_dates_data]).sort_index()

    filled_df.fillna(method='ffill', inplace=True)  # 이전 값으로 결측치 채우기
    filled_df.fillna(0, inplace=True)
    # 주어진 기간 동안의 데이터 추출
    filtered_data = filled_df.loc[start_date:end_date]

    return filtered_data

def cal_sma_ema(df):
  df['Change']=df['Close']-df['Open']
  df.loc[df['Change']>=0,'Volumecolor']=1
  df.loc[df['Change']<0,'Volumecolor']=0
  df[['Change','Volumecolor']]

  # 단순 이동평균
  df.loc[:, 'SMA_20'] = ta.sma(df['Close'],length=20)
  df.loc[:, 'SMA_60'] = ta.sma(df['Close'],length=60)

  # 지수 이동평균
  df.loc[:, 'EMA_20'] = ta.ema(df['Close'], 20)
  df.loc[:, 'EMA_60'] = ta.ema(df['Close'], 60)
  return df

def draw_candlechart(df):
  df['Change']=df['Close']-df['Open']
  df.loc[df['Change']>=0,'Volumecolor']=1
  df.loc[df['Change']<0,'Volumecolor']=0
  df[['Change','Volumecolor']]
  fig = make_subplots(rows=2,
                      cols=1,
                      shared_xaxes=True,
                      )

  fig = make_subplots(rows=2, cols=1)

  fig.add_trace(
      go.Candlestick(x=df.index,
                  open=df['Open'],
                  high=df['High'],
                  low=df['Low'],
                  close=df['Close'],
                  increasing_line_color= 'red', decreasing_line_color= 'blue',
                  name='주가'
                  ),
                  row=1, col=1
  )

  fig.add_trace(
      go.Bar(x=df.index, y=df['Volume'],
            marker=dict(color=df['Volumecolor'],colorscale='BlueRed'),
            name='거래량'),
      row=2, col=1
  )

  fig.update(layout_xaxis_rangeslider_visible=False)

  fig.show()

def draw_sma_ema(name,df):
  df = cal_sma_ema(df)

  fig = make_subplots(rows=2,
                      cols=1,
                      shared_xaxes=True,
                      vertical_spacing=0.1)

  fig = make_subplots(rows=2, cols=1)

  fig.add_trace(
      go.Candlestick(x=df.index,
                  open=df['Open'],
                  high=df['High'],
                  low=df['Low'],
                  close=df['Close'],
                  increasing_line_color= 'red', decreasing_line_color= 'blue',
                  name='주가'
                  ),
                  row=1, col=1
  )

  fig.add_trace(
      go.Scatter(x=df.index, y=df['SMA_20'],
                line=dict(color="#b66aa0"),
                name='SMA_20'),
      row=1, col=1
  )

  fig.add_trace(
      go.Scatter(x=df.index, y=df['SMA_60'],
                line=dict(color="#414b73"),
                  name='SMA_60'),
      row=1, col=1
  )

  fig.add_trace(
      go.Scatter(x=df.index, y=df['EMA_20'],
                line=dict(color="#8b8b8b"),
                  name='EMA_20'),
      row=1, col=1
  )


  fig.add_trace(
      go.Scatter(x=df.index, y=df['EMA_60'],
                line=dict(color="#e8d887"),
                  name='EMA_60'),
      row=1, col=1
  )

  fig.add_trace(
      go.Bar(x=df.index, y=df['Volume'],
            marker=dict(color=df['Volumecolor'],colorscale='BlueRed'),
            name='거래량'),
      row=2, col=1
  )

  fig.update(layout_xaxis_rangeslider_visible=False)
  fig.update_layout(title_text=name)

  fig.show()


# RSI
def draw_rsi(stock_name, stock_data):
  stock_data.loc[:, 'RSI_14'] = ta.rsi(stock_data['Close'], length=14)
  stock_data.loc[:, 'RSI_14'] = stock_data['RSI_14'].fillna(0)
  fig = plt.subplots(figsize=(10, 6), sharex=True)
  gs = gridspec.GridSpec(nrows=2, ncols=1, height_ratios=[2, 1])

  # 주가 나타내기
  ax1 = plt.subplot(gs[0])
  ax1 = stock_data['Close'].plot()
  ax1.set_xlabel('')
  ax1.axes.xaxis.set_ticks([])
  plt.title(stock_name)

  # RSI 나타내기
  ax2 = plt.subplot(gs[1])
  ax2 = stock_data['RSI_14'].plot(color='black', ylim=[0, 100])
  ax2.axhline(y=70, color='r', linestyle='-')
  ax2.axhline(y=30, color='r', linestyle='-')
  ax2.set_xlabel
  plt.subplots_adjust(wspace=0, hspace=0)

  plt.show()
  return stock_data

def draw_bbbands(stock_name, stock_data):
  band = ta.bbands(stock_data["Close"], length=20, std=2)
  bb = pd.concat([band[['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0']], stock_data['Close']], axis = 1)
  bb.columns = ['Lower Band', 'Mid Band', 'Upper Band', 'Close']
  bb.plot(figsize=(10, 6),
          color={
              'Upper Band': 'red',
              'Lower Band': 'blue',
              'Mid Band': 'green',
              'Close': 'black'
          })
  plt.title(stock_name)
  plt.show()

모든 반도체 종목에 대해 캔들차트 그리기

In [ ]:
name = '자람테크놀로지'
draw_sma_ema(name,get_stockdata(stock[name]))

In [ ]:
# for name, value in stock.items():
#   draw_bbbands(name,draw_rsi(name,draw_ema(name,draw_sma(name,match_dfshape(get_stockdata(value),'2021-01-01','2024-03-13')))))